In [ ]:
import os
import sys
sys.path.append('..')
sys.path.append('../source')

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import Electrostatics
import Polarization

from Utilities import write_xyz, KC
from AlphaGNN import Alpha

In [ ]:
def get_batch(key):
    multipoles = MULTIPOLES[key]
    elements_1, elements_2 = DATA[key]['elements']
    distance_matrices = DATA[key]['distance_matrices']
    coords_1, coords_2 = DATA[key]['coordinates']
    graph_1, graph_2 = DATA[key]['graphs']
    ratios_1, ratios_2 = RATIOS[key]
    return coords_1, coords_2, distance_matrices, elements_1, elements_2, graph_1, graph_2, ratios_1, ratios_2, multipoles

In [ ]:
RATIOS = np.load('RATIOS_DES5M.npy', allow_pickle=True).item()
MULTIPOLES = np.load('MULTIPOLES_DES5M.npy', allow_pickle=True).item()
DATA = np.load('DES5M.npy', allow_pickle=True).item()

In [ ]:
N_STEPS = 2
alpha_scaler = Alpha(n_steps=N_STEPS, load_weights=True)
FOLDER = ''

In [ ]:
for SMEAR in [-0.3, -0.35, -0.39]:
    ALPHAS, LR_TERMS, FIELDS, MU_IND, POL_TERMS = {}, {}, {}, {}, {}
    for idk, key in enumerate(DATA):
        coords_1, coords_2, distance_matrices, elements_1, elements_2, graph_1, graph_2, ratios_1, ratios_2, multipoles = get_batch(key)
        V_esp, F1, F2 = Electrostatics.esp_dimer(coords_1, coords_2, distance_matrices, multipoles, with_field=True)    
        alpha_coeffs_1, alpha_coeffs_2 = alpha_scaler(graph_1, 1)[None], alpha_scaler(graph_2, 1)[None]
        alphas_1, alphas_2 = alpha_coeffs_1 * ratios_1, alpha_coeffs_2 * ratios_2
        V_pol, mu_ind_1, mu_ind_2 = Polarization.ind_dimer(coords_1, coords_2, alphas_1, alphas_2, F1, F2, smear=SMEAR)
        ALPHAS[key] = (alphas_1, alphas_2)
        MU_IND[key] = (mu_ind_1, mu_ind_2)
        LR_TERMS[key] = (V_esp, V_pol)
    np.save(f'{FOLDER}ALPHAS_{N_STEPS}R', ALPHAS)
    np.save(f'{FOLDER}MU_IND_S{int(-SMEAR * 100)}{N_STEPS}R', MU_IND)
    np.save(f'{FOLDER}LR_TERMS_S{int(-SMEAR * 100)}{N_STEPS}R', LR_TERMS)
    